In [1]:
import os
import pandas as pd
import numpy as np
import requests
import json
import time
from IPython.display import clear_output
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
from django.db.models import Sum, Count
django.setup()
from tqdm import tqdm
from main_map.models import Organization, House, Metro, MetroStation, PolygonModel, RawLine, LineModel


/home/tserekh/jupnb/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def get_coord_and_time(lon_start, lat_start, lon_end, lat_end, mode):
    SERVER_URL_MSK_DRIVING = 'http://178.62.250.186:5000/route/v1/driving/'
    SERVER_URL_MSK_FOOT = 'http://178.62.250.186:5001/route/v1/foot/'
    SERVER_URL_PETER_DRIVING = 'http://178.62.250.186:5002/route/v1/driving/'
    SERVER_URL_PETER_FOOT = 'http://178.62.250.186:5003/route/v1/foot/'
    try:
        if lat_start > 58 and lat_el > 58:
            a = {
                'driving': SERVER_URL_PETER_DRIVING,
                'foot': SERVER_URL_PETER_FOOT,
                }[mode]
        else:
            a = {
                'driving': SERVER_URL_MSK_DRIVING,
                'foot': SERVER_URL_MSK_FOOT,
                }[mode]
    except KeyError as e:
        raise ValueError('Undefined mode: {}'.format(mode))


    b = str(lon_start)+','+str(lat_start)+';' + str(lon_end)+','+str(lat_end)
    c = '?annotations=duration'
    d = '&alternatives=false'
    e = '&overview=full&geometries=geojson'

    page_url =  a+b+c +d +e
    while True:
        try:
            page = requests.get(page_url)
            #print(page_url)
            break
        except:
            print('exception',end=' ')
            time.sleep(1)
    data = page.json()
    #print(data)
    coord_list = data['routes'][0]['geometry']['coordinates']
    time_list = data['routes'][0]['legs'][0]['annotation']['duration']
    return coord_list, time_list, data


In [3]:
print(RawLine.objects.all().count())
print(LineModel.objects.all().count())

6556104
0


In [4]:
# filter_ = {'att':'center','mode':'driving'}
# RawLine.objects.filter(**filter_).delete()
# LineModel.objects.filter(**filter_).delete()

In [5]:
#  filter_ = {'att':'center','mode':'driving'}
# LineModel.objects.filter(**filter_).delete()

In [6]:
# RawLine.objects.all().delete()
# LineModel.objects.all().delete()

In [7]:
# mode = 'foot'
# att = 'metro'
# duration_boarder = 15*60

att = 'center'
mode = 'driving'
duration_boarder = 100500*60

In [8]:
lat0,lon0 = 55.753215, 37.622504

In [9]:
print(55.753215 - 55.573575)
print(37.622504 - 37.285224)


0.17963999999999913
0.3372799999999998


In [10]:
#need_home_cols =  ['lat','lon','flat_num','square', 'sale_price']
need_home_cols =  ['lat','lon','flat_num', 'address']
objects = House.objects.filter().values().values_list(*need_home_cols)
df_habit = pd.DataFrame(list(objects), columns = need_home_cols)

In [11]:
#df_habit = pd.read_excel('Жилые дома с числом квартир и ценой v4.xlsx')
#df_habit = pd.read_excel('offices_base_ready.xlsx')

R_lat = 0.2
R_lon = 0.4

filt_lon = (np.abs(df_habit['lon'] - lon0) < R_lon).values
filt_lat = (np.abs(df_habit['lat'] - lat0) < R_lat).values
df_habit = df_habit[filt_lon&filt_lat]

In [12]:
need_home_cols =  ['lat','lon','station_name']
objects = Metro.objects.filter().values().values_list(*need_home_cols)
df_att = pd.DataFrame(list(objects), columns = need_home_cols)
df_att = df_att.rename(columns = {'station_name':'obj_name'})

In [13]:
att_mult = 2
filt_lon = np.abs(df_att['lon'] - lon0) < att_mult*R_lon
filt_lat = np.abs(df_att['lat'] - lat0) < att_mult*R_lat
df_att = df_att[filt_lon&filt_lat]#.iloc[0:1]

In [14]:
if att=='center':
    coords = [
        (55.748939, 37.608223),
        (55.753433, 37.612106),
        (55.756918, 37.616398),
        (55.758982, 37.621544),
        (55.759055, 37.626222),
        (55.756360, 37.630004),
        (55.750067, 37.631677),
        (55.752032, 37.624351),
        ]
    df_att = pd.DataFrame(coords, columns = ['lat','lon'])
    df_att['obj_name'] = 'center'

In [15]:
print(len(df_att))
print(len(df_habit))

8
44788


In [16]:
def get_dic(link):
    for i in range(100):
        content = requests.get(link).content
        dic = json.loads(content.decode())
        if 'message' not in dic.keys():
            break
        else:
            time.sleep(1)
            clear_output()
            print(dic)
    return dic

In [17]:
df_lines = pd.DataFrame()

In [18]:
from tqdm import tqdm

In [19]:
from functools import reduce


In [20]:
df_habit['lat'].mean(),df_habit['lon'].mean(),

(55.76435088784942, 37.644789094578904)

In [21]:
sd
area_lat_lon_rate = 2
print(len(df_habit))
for ind, habit_row in tqdm(df_habit.iterrows()):
    end_lat, end_lon = habit_row['lat'], habit_row['lon']

    coords = df_att[['lon','lat']].values
    

    pseudo_Rs = map(lambda x: (x[0]**2 + area_lat_lon_rate*(x[1]**2))**0.5,
                    (coords - np.array([habit_row['lon'],habit_row['lat']]))) 
    pseudo_Rs = list(pseudo_Rs)
    df_att['pseudo_R'] = pseudo_Rs
    N_atts = 6
    gb = df_att.groupby('obj_name')['pseudo_R'].min()
    if len(set(gb))>=N_atts:
        df_gb = gb.reset_index()
        pseudo_R_boarder = gb.sort_values().iloc[N_atts]
        df_att_temp = df_att[df_att['pseudo_R'] <= pseudo_R_boarder]
    else:
        df_att_temp = df_att.copy()
    
    coords = df_att_temp[['lon','lat']].values
    
    
    link = 'http://178.62.250.186:5001/table/v1/' + mode + '/'
    link += str(end_lon) + ',' + str(end_lat)
    for coord in coords:
        link += ';' + str(coord[0]) + ',' + str(coord[1])
    link += '?sources=0'
    dic = get_dic(link)

    durations = np.array(dic['durations'][0][1:])
    min_duration = np.min(durations)
    if min_duration > duration_boarder:
        #print('too long',end= ' ')
        continue
    att_row = df_att_temp[durations==min_duration].iloc[0]
    
    coords, times, data =  get_coord_and_time(att_row['lon'], att_row['lat'], habit_row['lon'], habit_row['lat'], mode)
    df_one_line = pd.DataFrame(coords, columns=['lon','lat'])
    df_line = df_one_line.iloc[:-1].rename(columns = {'lat':'lat0','lon':'lon0'})
    df_line['lat1'] = df_one_line['lat'].iloc[1:].values
    df_line['lon1'] = df_one_line['lon'].iloc[1:].values
    df_line['address'] = habit_row['address']
    df_line['weight'] = habit_row['flat_num']
    df_line['mode'] = mode
    df_line['att'] = att
    for row_line in df_line.T.to_dict().values():
        rawline = RawLine(**row_line)
        rawline.save()

NameError: name 'sd' is not defined

In [22]:
need_cols = ['lat0','lon0','lat1','lon1']
filter_dic = {}
# filter_dic['mode'] = mode
# filter_dic['att'] = att

filter_dic['mode'] = 'foot'
filter_dic['att'] = 'metro'

print(filter_dic)

{'mode': 'foot', 'att': 'metro'}


In [32]:
# LineModel.objects.all().delete()

In [35]:
for linemodel in tqdm(RawLine.objects.filter(**filter_dic).values(*need_cols).annotate(
                                                                            weight=Sum('weight'),
                                                                            n_parts=Count('lat0'))
                                                                                                      ):
    if linemodel['n_parts']!=1:
            
        linemodel['mode'] = filter_dic['mode'] 
        linemodel['att'] = filter_dic['att']
        linemodel = LineModel(**linemodel)
        linemodel.save()

100%|██████████| 125679/125679 [01:49<00:00, 1144.34it/s]


In [36]:
need_cols

['lat0', 'lon0', 'lat1', 'lon1']

In [37]:
df_data = pd.DataFrame(list(LineModel.objects.all().values(*['lat0','lon0']).annotate(count=Count('weight'))))
df_data = df_data[df_data['count']>1]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(100*np.array(lons), 100*np.array(lats))
plt.show()